In [1]:
import numpy as np
import pandas as pd
import redpitaya_scpi as scpi
import time
import datetime
import os
import json
import joblib
from scipy.signal import welch
from scipy import stats
import sys  # Add this if not already imported


# Function to load configurations from config.json
def load_config(config_path='config.json'):
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    required_keys = ["ip_address", "decimation_factor", "scaler_path", "model_path"]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config


# Function to compute statistical features (copied from Script 2)
def compute_statistical_features(data_array):
    """
    Compute statistical features from a data array.
    """
    features = {}
    try:
        features['entropy'] = stats.entropy(np.abs(data_array))
        features['skewness'] = stats.skew(data_array)
        features['interquartile_range'] = np.percentile(data_array, 75) - np.percentile(data_array, 25)
        features['kurtosis'] = stats.kurtosis(data_array)
        features['percentile_75'] = np.percentile(data_array, 75)
        features['range'] = np.ptp(data_array)
        features['maximum'] = np.max(data_array)
        features['median'] = np.median(data_array)
        features['percentile_90'] = np.percentile(data_array, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(data_array - np.mean(data_array)))
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
            'percentile_75', 'range', 'maximum', 'median',
            'percentile_90', 'mean_absolute_deviation']}
    return features


# Function to process live data and classify using the KNN model
def classify_live_data(signal, scaler, model, decimation_factor):
    """
    Process live data from Red Pitaya and classify it using the trained model.
    """
    # Remove DC component
    signal = signal - np.mean(signal)

    # Compute Welch power spectrum (do not normalize)
    frequencies, power_spectrum = welch(signal, fs=125e6 / decimation_factor, nperseg=1024)
    # Do not normalize power_spectrum here
    # power_spectrum = power_spectrum / np.sum(power_spectrum)  # This line is removed

    # Compute statistical features using the same method as in Script 2
    features = compute_statistical_features(power_spectrum)
    feature_array = np.array(list(features.values())).reshape(1, -1)
    
    # Scale features and classify
    scaled_features = scaler.transform(feature_array)
    probabilities = model.predict_proba(scaled_features)[0]
    
    # Map probabilities to device labels
    device_labels = model.classes_
    timestamp = datetime.datetime.now().isoformat()
    classification_result = {'timestamp': timestamp}
    classification_result.update({label: prob for label, prob in zip(device_labels, probabilities)})
    
    return classification_result


# Main data collection and classification function
def collect_and_classify(config):
    # Extract configurations
    ip_address = config['ip_address']
    decimation_factor = config['decimation_factor']
    scaler_path = config['scaler_path']
    model_path = config['model_path']
    
    # Load scaler and model
    scaler = joblib.load(scaler_path)
    model = joblib.load(model_path)
    
    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(ip_address)
        print(f"Connected to Red Pitaya at {ip_address}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {ip_address}: {e}")
        return
    
    try:
        while True:
            # Configure Red Pitaya
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                time.sleep(1)  # Wait for data capture
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(2)
                continue  # Skip to next iteration
            
            # Capture data from channel 1
            try:
                rp_s.tx_txt('ACQ:SOUR1:DATA?')
                raw_data_ch1 = rp_s.rx_txt().strip('{}\n\r').split(',')
                signal_ch1 = np.array([float(x) for x in raw_data_ch1])
                # Remove DC component (already done in classify_live_data)
                # signal_ch1 -= np.mean(signal_ch1)
            except Exception as e:
                print(f"Error acquiring data from channel 1: {e}")
                time.sleep(1)
                continue  # Skip to next iteration
            
            # Classify live data
            if signal_ch1.size > 0:
                try:
                    result = classify_live_data(signal_ch1, scaler, model, decimation_factor)
                    print(f"Classification result: {result}")
                except Exception as e:
                    print(f"Error classifying live data: {e}")
            else:
                print("No valid data captured from channel 1.")
            
            # Optional: control loop frequency
            time.sleep(1)  # Adjust the delay as needed
    except KeyboardInterrupt:
        print("\nStopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Entry point
if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)
    
    collect_and_classify(config)


Connected to Red Pitaya at 192.168.8.214
Classification result: {'timestamp': '2024-12-02T15:49:12.661147', 'empty': 1.0, 'ipad': 0.0, 'lamp': 0.0, 'mac': 0.0}
Classification result: {'timestamp': '2024-12-02T15:49:15.846520', 'empty': 1.0, 'ipad': 0.0, 'lamp': 0.0, 'mac': 0.0}
Classification result: {'timestamp': '2024-12-02T15:49:18.272632', 'empty': 1.0, 'ipad': 0.0, 'lamp': 0.0, 'mac': 0.0}

Stopped by user.


# dos

In [7]:
import requests  # Add requests library import
import numpy as np
import pandas as pd
import redpitaya_scpi as scpi
import time
import datetime
import os
import json
import joblib
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler


# Function to load configurations from config.json
def load_config(config_path='config.json'):
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    required_keys = ["ip_address", "decimation_factor", "scaler_path", "model_path"]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config


# Function to compute statistical features (copied from Script 2)
def compute_statistical_features(data_array):
    features = {}
    try:
        features['entropy'] = stats.entropy(np.abs(data_array))
        features['skewness'] = stats.skew(data_array)
        features['interquartile_range'] = np.percentile(data_array, 75) - np.percentile(data_array, 25)
        features['kurtosis'] = stats.kurtosis(data_array)
        features['percentile_75'] = np.percentile(data_array, 75)
        features['range'] = np.ptp(data_array)
        features['maximum'] = np.max(data_array)
        features['median'] = np.median(data_array)
        features['percentile_90'] = np.percentile(data_array, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(data_array - np.mean(data_array)))
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
            'percentile_75', 'range', 'maximum', 'median',
            'percentile_90', 'mean_absolute_deviation']}
    return features


# Function to process live data and classify using the KNN model
def classify_live_data(signal, scaler, model, decimation_factor):
    # Remove DC component
    signal = signal - np.mean(signal)

    # Compute Welch power spectrum
    frequencies, power_spectrum = welch(signal, fs=125e6 / decimation_factor, nperseg=1024)

    # Compute statistical features
    features = compute_statistical_features(power_spectrum)
    feature_array = np.array(list(features.values())).reshape(1, -1)

    # Scale features and classify
    scaled_features = scaler.transform(feature_array)
    probabilities = model.predict_proba(scaled_features)[0]

    # Map probabilities to device labels
    device_labels = model.classes_
    timestamp = datetime.datetime.now().isoformat()
    classification_result = {'timestamp': timestamp}
    classification_result.update({label: prob for label, prob in zip(device_labels, probabilities)})

    return classification_result


# Function to send data to the API
def send_data_to_api(live_results_row):
    """
    Sends the device states as a JSON payload to the specified API endpoint.
    
    Parameters:
    - live_results_row: Dictionary containing classification probabilities.
    """
    API_ENDPOINT = 'https://thpjgw8n89.execute-api.us-east-1.amazonaws.com/prod/updateDeviceState'
    
    # Map devices
    devices = [
        {'deviceId': 'device1', 'on': bool(live_results_row['mac'] > 0.5)},
        {'deviceId': 'device2', 'on': bool(live_results_row['lamp'] > 0.5)},
        {'deviceId': 'device3', 'on': bool(live_results_row['ipad'] > 0.5)}
    ]
    # devices = [
    #     {'deviceId': 'device1', 'on': bool(0)},
    #     {'deviceId': 'device2', 'on': bool(live_results_row['lamp'] > 0.5)},
    #     {'deviceId': 'device3', 'on': bool(live_results_row['ipad'] > 0.5)}
    # ]
    
    # Prepare the request payload
    payload = {
        'devices': devices
    }
    print(payload)
    
    # Send the POST request
    try:
        response = requests.post(API_ENDPOINT, json=payload)
        response.raise_for_status()
        print('Data sent successfully:', response.status_code)
        print('Response:', response.json())
    except requests.exceptions.RequestException as e:
        print(f"Failed to send data to API: {e}")


# Main data collection and classification function
def collect_and_classify(config):
    # Extract configurations
    ip_address = config['ip_address']
    decimation_factor = config['decimation_factor']
    scaler_path = config['scaler_path']
    model_path = config['model_path']

    # Load scaler and model
    scaler = joblib.load(scaler_path)
    model = joblib.load(model_path)

    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(ip_address)
        print(f"Connected to Red Pitaya at {ip_address}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {ip_address}: {e}")
        return

    try:
        while True:
            # Configure Red Pitaya
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                time.sleep(1)  # Wait for data capture
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(2)
                continue  # Skip to next iteration

            # Capture data from channel 1
            try:
                rp_s.tx_txt('ACQ:SOUR1:DATA?')
                raw_data_ch1 = rp_s.rx_txt().strip('{}\n\r').split(',')
                signal_ch1 = np.array([float(x) for x in raw_data_ch1])
            except Exception as e:
                print(f"Error acquiring data from channel 1: {e}")
                time.sleep(1)
                continue  # Skip to next iteration

            # Classify live data
            if signal_ch1.size > 0:
                try:
                    result = classify_live_data(signal_ch1, scaler, model, decimation_factor)
                    print(f"Classification result: {result}")

                    # Send the classification result to the API
                    send_data_to_api(result)
                except Exception as e:
                    print(f"Error classifying live data or sending to API: {e}")
            else:
                print("No valid data captured from channel 1.")

            # Optional: control loop frequency
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nStopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Entry point
if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)

    collect_and_classify(config)


Connected to Red Pitaya at 192.168.8.214
Classification result: {'timestamp': '2024-12-02T17:53:06.159865', 'empty': 1.0, 'ipad': 0.0, 'lamp': 0.0, 'mac': 0.0}
{'devices': [{'deviceId': 'device1', 'on': False}, {'deviceId': 'device2', 'on': False}, {'deviceId': 'device3', 'on': False}]}
Data sent successfully: 200
Response: {'message': 'All device states updated successfully.', 'updatedDevices': [{'deviceId': 'device1', 'updatedAttributes': {'on': False}}, {'deviceId': 'device2', 'updatedAttributes': {'on': False}}, {'deviceId': 'device3', 'updatedAttributes': {'on': False}}]}
Classification result: {'timestamp': '2024-12-02T17:53:09.205363', 'empty': 1.0, 'ipad': 0.0, 'lamp': 0.0, 'mac': 0.0}
{'devices': [{'deviceId': 'device1', 'on': False}, {'deviceId': 'device2', 'on': False}, {'deviceId': 'device3', 'on': False}]}
Data sent successfully: 200
Response: {'message': 'All device states updated successfully.', 'updatedDevices': [{'deviceId': 'device1', 'updatedAttributes': {'on': False

In [ ]:
import numpy as np
import pandas as pd
import redpitaya_scpi as scpi
import time
import datetime
import os
import json
import joblib
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler


# Function to load configurations from config.json
def load_config(config_path='config.json'):
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    required_keys = ["ip_address", "decimation_factor", "scaler_path", "model_path"]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config


# Function to compute statistical features
def compute_statistical_features(signal):
    """
    Compute statistical features from the Welch power spectrum of a signal.
    """
    try:
        features = {
            'entropy': np.sum(-signal * np.log2(signal + 1e-12)),  # Avoid log(0)
            'skewness': pd.Series(signal).skew(),
            'interquartile_range': np.percentile(signal, 75) - np.percentile(signal, 25),
            'kurtosis': pd.Series(signal).kurtosis(),
            'percentile_75': np.percentile(signal, 75),
            'range': np.ptp(signal),
            'maximum': np.max(signal),
            'median': np.median(signal),
            'percentile_90': np.percentile(signal, 90),
            'mean_absolute_deviation': np.mean(np.abs(signal - np.mean(signal)))
        }
    except Exception as e:
        print(f"Error computing features: {e}")
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis', 'percentile_75',
            'range', 'maximum', 'median', 'percentile_90', 'mean_absolute_deviation']}
    
    return features


# Function to process live data and classify using the KNN model
def classify_live_data(signal, scaler, model, decimation_factor):
    """
    Process live data from Red Pitaya and classify it using the trained model.
    """
    # Compute Welch power spectrum
    frequencies, power_spectrum = welch(signal, fs=125e6 / decimation_factor, nperseg=1024)
    power_spectrum = power_spectrum / np.sum(power_spectrum)  # Normalize to get probabilities
    
    # Compute statistical features
    features = compute_statistical_features(power_spectrum)
    feature_array = np.array(list(features.values())).reshape(1, -1)
    
    # Scale features and classify
    scaled_features = scaler.transform(feature_array)
    probabilities = model.predict_proba(scaled_features)[0]
    
    # Map probabilities to device labels
    device_labels = model.classes_
    timestamp = datetime.datetime.now().isoformat()
    classification_result = {'timestamp': timestamp}
    classification_result.update({label: prob for label, prob in zip(device_labels, probabilities)})
    
    return classification_result


# Main data collection and classification function
def collect_and_classify(config):
    # Extract configurations
    ip_address = config['ip_address']
    decimation_factor = config['decimation_factor']
    scaler_path = config['scaler_path']
    model_path = config['model_path']
    
    # Load scaler and model
    scaler = joblib.load(scaler_path)
    model = joblib.load(model_path)
    
    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(ip_address)
        print(f"Connected to Red Pitaya at {ip_address}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {ip_address}: {e}")
        return
    
    try:
        while True:
            # Configure Red Pitaya
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                time.sleep(1)  # Wait for data capture
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(2)
                continue  # Skip to next iteration
            
            # Capture data from channel 1
            try:
                rp_s.tx_txt('ACQ:SOUR1:DATA?')
                raw_data_ch1 = rp_s.rx_txt().strip('{}\n\r').split(',')
                signal_ch1 = np.array([float(x) for x in raw_data_ch1])
                signal_ch1 -= np.mean(signal_ch1)  # Remove DC component
            except Exception as e:
                print(f"Error acquiring data from channel 1: {e}")
                time.sleep(1)
                continue  # Skip to next iteration
            
            # Classify live data
            if signal_ch1.size > 0:
                try:
                    result = classify_live_data(signal_ch1, scaler, model, decimation_factor)
                    print(f"Classification result: {result}")
                except Exception as e:
                    print(f"Error classifying live data: {e}")
            else:
                print("No valid data captured from channel 1.")
            
            # Optional: control loop frequency
            time.sleep(1)  # Adjust the delay as needed
    except KeyboardInterrupt:
        print("\nStopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Entry point
if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)
    
    collect_and_classify(config)


Connected to Red Pitaya at 192.168.8.214
Classification result: {'timestamp': '2024-12-02T15:09:57.324487', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:01.138478', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:05.189753', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:10.092226', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:15.218850', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:19.080551', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:23.643194', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:27.656702', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:31.632490', 'ipad': 0.0, 'lamp': 1.0}
Classification result: {'timestamp': '2024-12-02T15:10:35.393793', 'ipad': 0.0, 'lamp': 1.0}

Stopped by user.
